# Assignment: Creating Interactive Dashboard with User Subscription Dataset

> Keerti

## Reading the Data

In [16]:
import pandas as pd

# Assuming the dataset is stored in a CSV file named 'subscriptions.csv'
df = pd.read_csv('/content/NioPracticeUserbase.csv', index_col=0)

# Display the dataframe
df

,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration
User ID,,,,,,,,,
1,Basic,10,15-01-22,10-06-23,United States,28,Male,Smartphone,1 Month
2,Premium,15,05-09-21,22-06-23,Canada,35,Female,Tablet,1 Month
3,Standard,12,28-02-23,27-06-23,United Kingdom,42,Male,Smart TV,1 Month
4,Standard,12,10-07-22,26-06-23,Australia,51,Female,Laptop,1 Month
5,Basic,10,01-05-23,28-06-23,Germany,33,Male,Smartphone,1 Month
...,...,...,...,...,...,...,...,...,...
2496,Premium,14,25-07-22,12-07-23,Spain,28,Female,Smart TV,1 Month
2497,Basic,15,04-08-22,14-07-23,Spain,33,Female,Smart TV,1 Month
2498,Standard,12,09-08-22,15-07-23,United States,38,Male,Laptop,1 Month


## Data Preprocessing

In [17]:
print("Null values in the dataset:")
print(df.isnull().sum())
print("\nData types of the columns:")
print(df.dtypes)

Null values in the dataset:
Subscription Type    0
Monthly Revenue      0
Join Date            0
Last Payment Date    0
Country              0
Age                  0
Gender               0
Device               0
Plan Duration        0
dtype: int64

Data types of the columns:
Subscription Type    object
Monthly Revenue       int64
Join Date            object
Last Payment Date    object
Country              object
Age                   int64
Gender               object
Device               object
Plan Duration        object
dtype: object


In [ ]:
# Convert 'Join Date' and 'Last Payment Date' columns to datetime
df['Join Date'] = pd.to_datetime(df['Join Date'])
df['Last Payment Date'] = pd.to_datetime(df['Last Payment Date'])

## Data Visualization

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go

In [18]:
# Initialize the Dash app
app = dash.Dash(__name__)

# User Demographics Visualization
user_demographics_layout = html.Div([
    html.Div([
        html.H2("User Demographics"),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country, 'value': country} for country in df['Country'].unique()],
            value='United States',
            clearable=False
        ),
        dcc.Graph(id='user-demographics-graph')
    ], className="box")
])

@app.callback(
    Output('user-demographics-graph', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_user_demographics_graph(selected_country):
    filtered_df = df[df['Country'] == selected_country]
    gender_counts = filtered_df['Gender'].value_counts()

    fig = go.Figure(go.Bar(
        x=gender_counts.index,
        y=gender_counts.values,
        marker_color='blue'
    ))

    fig.update_layout(
        title="Number of Users by Gender",
        xaxis_title="Gender",
        yaxis_title="Number of Users",
        showlegend=False
    )

    return fig

# Subscription Overview Visualization
subscription_overview_layout = html.Div([
    html.Div([
        html.H2("Subscription Overview"),
        dcc.Dropdown(
            id='duration-dropdown',
            options=[{'label': duration, 'value': duration} for duration in df['Plan Duration'].unique()],
            value='1 Month',
            clearable=False
        ),
        dcc.Graph(id='subscription-overview-graph')
    ], className="box")
])

@app.callback(
    Output('subscription-overview-graph', 'figure'),
    [Input('duration-dropdown', 'value')]
)
def update_subscription_overview_graph(selected_duration):
    duration_counts = df[df['Plan Duration'] == selected_duration]['Subscription Type'].value_counts()

    fig = go.Figure(go.Pie(
        labels=duration_counts.index,
        values=duration_counts.values
    ))

    fig.update_layout(
        title="Proportion of Each Subscription Type",
        showlegend=True
    )

    return fig

# Monthly Revenue Trend Visualization
monthly_revenue_layout = html.Div([
    html.Div([
        html.H2("Monthly Revenue Trend"),
        dcc.RadioItems(
            id='subscription-type-buttons',
            options=[
                {'label': 'Basic', 'value': 'Basic'},
                {'label': 'Standard', 'value': 'Standard'},
                {'label': 'Premium', 'value': 'Premium'}
            ],
            value='Basic',
            labelStyle={'display': 'inline-block'}
        ),
        dcc.Graph(id='monthly-revenue-graph')
    ], className="box")
])

@app.callback(
    Output('monthly-revenue-graph', 'figure'),
    [Input('subscription-type-buttons', 'value')]
)
def update_monthly_revenue_graph(selected_subscription_type):
    filtered_df = df[df['Subscription Type'] == selected_subscription_type]
    monthly_revenue = filtered_df.groupby(pd.to_datetime(filtered_df['Join Date']).dt.to_period('M'))['Monthly Revenue'].sum()

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=monthly_revenue.index.astype(str),
        y=monthly_revenue.values,
        mode='lines+markers',
        name='Total Monthly Revenue'
    ))

    fig.update_layout(
        title="Total Monthly Revenue Over Time",
        xaxis_title="Month",
        yaxis_title="Total Revenue",
        showlegend=True
    )

    return fig

# Define the layout of the app
app.layout = html.Div([
    html.Div([
        user_demographics_layout,
        subscription_overview_layout
    ], className="top-row"),
    html.Div([
        monthly_revenue_layout
    ], className="bottom-row")
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>